In [1]:
using IntervalArithmetic,IntervalBoxes,IntervalArithmetic.Symbols,Plots
using Statistics

In [3]:
x1 = 1/3
x2 = 2/3
h = 1.0
g = 9.8
function calculate_expression(y1::Interval{T}, y2::Interval{T}) where T
    term1 = sqrt(x1^2 + (y1 - h)^2) / sqrt(2 * g * (h - y1))
    term2 = sqrt((x1 - x2)^2 + (y1 - y2)^2) / (sqrt(2 * g * (h - y1)) + sqrt(2 * g * (h - y2)))
    term3 = sqrt(y2^2 + (x2 - h)^2) / (sqrt(2 * g * (h - y2)) + sqrt(2 * g * h))

    return 2*(term1 + term2 + term3)
end

calculate_expression (generic function with 1 method)

In [6]:

# Define the branch-and-bound function
function calculate_branch_bound(f, Y1::Interval{T}, Y2::Interval{T}, N) where T
    interval_lists = [[Y1, Y2]]
    upper_bound = sup(f(Y1..., Y2...))
    upper_bounds = [upper_bound]
    working_list = [(Y1..., Y2...)]

    for i in 1:N
        if isempty(working_list)
            break
        end
        
        # Pop the first interval pair
        (current_Y1, current_Y2) = popfirst!(working_list)

        # Update the upper bound
        upper_bound = min(upper_bound, sup(f(interval(mid(current_Y1))..., interval(mid(current_Y2))...)))
        # Check if we can bisect
        if inf(f(current_Y1..., current_Y2...)) <= upper_bound
            if(diam(current_Y1)>= diam(current_Y2))
                if(diam(current_Y1)>=0.0001)
                    S = bisect(current_Y1)
                    push!(working_list, (S[1],current_Y2), (S[2],current_Y2))
                else
                    push!(working_list,(current_Y1,current_Y2))
                end
            else
                if(diam(current_Y2)>=0.0001)
                    S1 = bisect(current_Y2)
                    push!(working_list, (current_Y1,S1[1]), (current_Y1,S1[2]))
                else
                    push!(working_list,(current_Y1,current_Y2))
                end
            end
        end

        # Store the current intervals and upper bounds
        #push!(interval_lists, copy(working_list))
        push!(upper_bounds, upper_bound)
    end

    return working_list, upper_bounds
end



calculate_branch_bound (generic function with 1 method)

In [7]:
N = 10000
y1_interval = interval(0.0, 1)
y2_interval = interval(0.0, 1)
working_list, minimas = calculate_branch_bound(calculate_expression, y1_interval, y2_interval, N)
#println(working_list)
# midpoints = [(mean(y1), mean(y2)) for (y1, y2) in working_list]
# println(midpoints)
# Calculate midpoints for y1 and y2 separately
# y1_mid = [mid(y[1]) for y in working_list]  
# y2_mid = [mid(y[2]) for y in working_list]  

#println(y1_mid)
#@show(y1_mid)
# # Calculate radii (error) for y1 and y2 separately
# y1_rad = [diam(y[1]) / 2 for y in Ys]  # Diameter divided by 2 for the first interval
# y2_rad = [diam(y[2]) / 2 for y in Ys]  

# Create arrays for x1 and x2
# x11 = 0.333 * ones(size(y1_mid))  # Constant x1
# x21 = 0.667 * ones(size(y2_mid))  # Constant x2

# Plot y1 and y2 with error bars
#plot(x11, y1_mid, legend=false, xlabel="x", ylabel="y", xlimits=(0, 1), ylimits=(0, 1))

# Plot y1 and y2 with error bars
# plot(x11, y1, yerror=y1_rad, legend=false,xlimits=(0, 1), ylimits=(0, 1))
# plot(x21, y2, yerror=y2_rad, legend=false,xlimits=(0, 1), ylimits=(0, 1))

(Tuple{Interval{Float64}, Interval{Float64}}[([0.365234, 0.367188]_com, [0.154296, 0.156251]_com), ([0.367187, 0.369141]_com, [0.140624, 0.142579]_com), ([0.367187, 0.369141]_com, [0.142578, 0.144532]_com), ([0.36914, 0.371094]_com, [0.140624, 0.142579]_com), ([0.36914, 0.371094]_com, [0.142578, 0.144532]_com), ([0.367187, 0.369141]_com, [0.144531, 0.146485]_com), ([0.367187, 0.369141]_com, [0.146484, 0.148438]_com), ([0.36914, 0.371094]_com, [0.144531, 0.146485]_com), ([0.36914, 0.371094]_com, [0.146484, 0.148438]_com), ([0.371093, 0.373047]_com, [0.140624, 0.142579]_com)  …  ([0.365234, 0.366211]_com, [0.148437, 0.150391]_com), ([0.36621, 0.367188]_com, [0.148437, 0.150391]_com), ([0.365234, 0.366211]_com, [0.15039, 0.152344]_com), ([0.36621, 0.367188]_com, [0.15039, 0.152344]_com), ([0.363281, 0.364258]_com, [0.152343, 0.154297]_com), ([0.364257, 0.365235]_com, [0.152343, 0.154297]_com), ([0.363281, 0.364258]_com, [0.154296, 0.156251]_com), ([0.364257, 0.365235]_com, [0.154296, 0.15

In [11]:
s = calculate_expression(interval(0.36621, 0.367188), interval(0.152343, 0.154297))

[0.596001, 0.597564]_com_NG